In [5]:
import pandas as pd

In [7]:
df_genderlabel_added = pd.read_csv('./full_data+genderlabel_added.csv', encoding='utf-8-sig')
df_genderlabel_added.drop(columns=['hashtag','raw_text'], inplace=True)
df_genderlabel_added

,_id,followers,gender,location,re_followers,re_gender,re_location,re_mid,re_time,re_uid,time,uid,text,gender_label
0,4778778284788623,165000,m,广东,-1,NaN,NaN,-1,NaN,-1,"2022/06/10, 11:49:51",7319642154,一帮大男人，摸人家女的，还打人家几个女的！,2.0
1,4778788178625613,682000,m,广东,-1,NaN,NaN,-1,NaN,-1,"2022/06/10, 12:29:10",6420207784,调戏美女被拒，恼羞成怒对人进行殴打！这帮人太嚣张了！！,2.0
2,4778791911558944,544000,m,湖北,-1,NaN,NaN,-1,NaN,-1,"2022/06/10, 12:44:00",6078562289,简直是丢男人的脸！摸人家女的被骂有病就动手打人，还一群男人一起动手……→_→评论配图,1.0
3,4778793132884451,165000,m,广东,-1,NaN,NaN,-1,NaN,-1,"2022/06/10, 12:48:50",7319642154,当众调戏美女遭拒，竟然恼羞成怒…,2.0
4,4778794802745132,98000,f,广东,-1,NaN,NaN,-1,NaN,-1,"2022/06/10, 12:55:29",5752980584,俩男子索要美女微信不成，竟对其大打出手！黑社会吗？？,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
724441,4786528877022355,2,m,广西,-1,NaN,NaN,-1,NaN,-1,"2022/07/01, 21:07:56",7558373492,那四个女生到底去哪儿了？媒体居然连个身边人都找不到，不免让人担心吧,2.0
724442,4786535020889424,38,f,河南,695,f,甘肃,4786489579275197,"2022/07/01, 18:31:46",5359744849,"2022/07/01, 21:32:21",6229065644,NaN,NaN
724443,4786538246574989,12,f,北京,12,f,北京,4781467517585807,"2022/06/17, 21:55:54",6130628947,"2022/07/01, 21:45:10",6130628947,等后续等后续！！！！没有忘,2.0
724444,4786545335207567,42,m,浙江,209,f,山西,4782381833130758,"2022/06/20, 10:29:04",2738220895,"2022/07/01, 22:13:20",7714950616,NaN,NaN


In [17]:
df_has_text = df_genderlabel_added[~df_genderlabel_added['text'].isna()] # select NOT NaN rows
chi_texts = df_has_text['text'].tolist()

In [53]:
def partition(l, n):
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [26]:
chi_texts_chunks = list(partition(chi_texts, 10000))

In [37]:
from pygtrans import Translate
client = Translate()

In [ ]:
eng_texts = []

In [85]:
for chunk_index in range(len(chi_texts_chunks)-1, len(chi_texts_chunks)):
    
    ten_thousands_texts = []
    res_ls = client.translate(chi_texts_chunks[chunk_index], target='en') # one request
    
    for res in res_ls:
        if isinstance(res, type(None)):
            print('Failed once')
        ten_thousands_texts.append(res.translatedText.replace('&#39;', "'").replace('&#34;', '"').replace('&#38;', '&').replace('&#60;', '<').replace('&#62;', '>'))
    
    if chunk_index != len(chi_texts_chunks)-1 and len(ten_thousands_texts) != 10000:
        print('Alert: call '+str(chunk_index)+' returned texts less than 10000')
        break
    
    # OK to append tp eng_texts
    eng_texts.append(ten_thousands_texts)
    print("Length of eng_texts: ", len(eng_texts))

Length of eng_texts:  65


In [93]:
def flatten(l):
    return [item for sublist in l for item in sublist]

[1, 2, 3, 4]

In [94]:
eng_texts_flatten = flatten(eng_texts)
len(eng_texts_flatten)

645864

In [95]:
df_has_text['text_eng'] = eng_texts_flatten

C:\Users\24029\AppData\Local\Temp\ipykernel_14492\3976646338.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_has_text['text_eng'] = eng_texts_flatten


In [106]:
for i in range(0, len(eng_texts_flatten)):
    eng_texts_flatten[i] = eng_texts_flatten[i].replace('&apos;', "'").replace('&quot;', '"').replace('&amp;', '&').replace('&lt;', '<').replace('&gt;', '>')

In [110]:
df_has_text

,_id,followers,gender,location,re_followers,re_gender,re_location,re_mid,re_time,re_uid,time,uid,text,gender_label,text_eng
0,4778778284788623,165000,m,广东,-1,NaN,NaN,-1,NaN,-1,"2022/06/10, 11:49:51",7319642154,一帮大男人，摸人家女的，还打人家几个女的！,2.0,A group of big men touched other women's women...
1,4778788178625613,682000,m,广东,-1,NaN,NaN,-1,NaN,-1,"2022/06/10, 12:29:10",6420207784,调戏美女被拒，恼羞成怒对人进行殴打！这帮人太嚣张了！！,2.0,"Molesting a beautiful woman was rejected, and ..."
2,4778791911558944,544000,m,湖北,-1,NaN,NaN,-1,NaN,-1,"2022/06/10, 12:44:00",6078562289,简直是丢男人的脸！摸人家女的被骂有病就动手打人，还一群男人一起动手……→_→评论配图,1.0,What a disgrace to a man! Those who touch othe...
3,4778793132884451,165000,m,广东,-1,NaN,NaN,-1,NaN,-1,"2022/06/10, 12:48:50",7319642154,当众调戏美女遭拒，竟然恼羞成怒…,2.0,Molesting a beautiful woman in public was reje...
4,4778794802745132,98000,f,广东,-1,NaN,NaN,-1,NaN,-1,"2022/06/10, 12:55:29",5752980584,俩男子索要美女微信不成，竟对其大打出手！黑社会吗？？,0.0,Two men beat her up when they failed to ask fo...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
724439,4786515714509930,1,f,广东,-1,NaN,NaN,-1,NaN,-1,"2022/07/01, 20:15:38",7746467515,这么多天过去了，四个女孩的现状于情于理都该被披露，因为社会关心她们，她们的健康情况也是事件核...,2.0,"After so many days, the current situation of t..."
724440,4786527463279012,7404,m,山东,-1,NaN,NaN,-1,NaN,-1,"2022/07/01, 21:02:19",7024438800,日照,2.0,sunshine
724441,4786528877022355,2,m,广西,-1,NaN,NaN,-1,NaN,-1,"2022/07/01, 21:07:56",7558373492,那四个女生到底去哪儿了？媒体居然连个身边人都找不到，不免让人担心吧,2.0,Where did those four girls go? The media can't...
724443,4786538246574989,12,f,北京,12,f,北京,4781467517585807,"2022/06/17, 21:55:54",6130628947,"2022/07/01, 21:45:10",6130628947,等后续等后续！！！！没有忘,2.0,"Wait for the follow-up, wait for the follow-up..."


In [111]:
df_has_text.to_csv('translated_text_with_other_data.csv', index=False, encoding='utf-8-sig')